In [103]:
!pip list

Package                  Version
------------------------ ---------------
absl-py                  1.0.0
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
asn1crypto               0.24.0
astor                    0.8.1
astunparse               1.6.3
attrs                    21.4.0
backcall                 0.2.0
beautifulsoup4           4.11.1
bleach                   5.0.0
boto3                    1.33.13
botocore                 1.33.13
cachetools               5.0.0
certifi                  2021.10.8
cffi                     1.15.0
charset-normalizer       2.0.12
cloudpickle              2.2.1
copulas                  0.9.0
cryptography             2.1.4
ctgan                    0.7.3
cycler                   0.11.0
debugpy                  1.6.0
decorator                5.1.1
deepecho                 0.4.1
defusedxml               0.7.1
dm-tree                  0.1.7
ecos                     2.0.13
entrypoints              0.4
et-xmlfile               1.1.0
Faker       

In [105]:

import tensorflow as tf
print(tf.__version__)

1.15.5


In [106]:

import sys
print(sys.version)

3.7.5 (default, Dec  9 2021, 17:04:37) 
[GCC 8.4.0]


In [1]:

import pandas as pd
import os
import numpy as np

In [2]:

from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored

import csv
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.ensemble import ExtraSurvivalTrees
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

In [3]:
xl = pd.read_excel('/project/kc20230915001/src/STOMACH13_V7 Younger.xlsx')

In [4]:
xl.to_csv('/project/kc20230915001/src/data_young.csv', index=False)

In [5]:

df = pd.read_csv('/project/kc20230915001/src/data_young.csv')

### Data encoding

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [7]:
df.columns

Index(['SEX', 'TCODE', 'MCODE_GRP', 'GRADE', 'AGE', 'T_SIZE', 'AJCC7_STAGE',
       'G1E_HGHT', 'G1E_WGHT', 'G1E_BMI', 'G1E_WSTC', 'G1E_BP_SYS',
       'G1E_BP_DIA', 'G1E_URN_PROT', 'G1E_HGB', 'G1E_FBS', 'G1E_TOT_CHOL',
       'G1E_SGOT', 'G1E_SGPT', 'G1E_GGT', 'G1E_TG', 'G1E_HDL', 'G1E_LDL',
       'G1E_CRTN', 'G1E_GFR', 'Q_SMK_YN', 'Q_DRK_FRQ_V09N', 'Q_DRK_AMT_V09N',
       'Q_PA_VD', 'Q_PA_MD', 'Q_PA_WALK', 'AF', 'CKD', 'COPD', 'DM', 'DVT',
       'DYS', 'HF', 'hyper', 'LD', 'MI', 'obesity', 'Obesity', 'stroke',
       '5year_survival_time(day)', '5year_death', '3year_survival_time(day)',
       '3year_death', '1year_survival_time(day)', '1year_death'],
      dtype='object')

In [8]:
X = df[['SEX', 'TCODE', 'MCODE_GRP', 'GRADE', 'AGE', 'T_SIZE', 'AJCC7_STAGE',
        'G1E_HGHT', 'G1E_WGHT', 'G1E_BMI', 'G1E_WSTC', 'G1E_BP_SYS',
        'G1E_BP_DIA', 'G1E_URN_PROT', 'G1E_HGB', 'G1E_FBS', 'G1E_TOT_CHOL',
        'G1E_SGOT', 'G1E_SGPT', 'G1E_GGT', 'G1E_TG', 'G1E_HDL', 'G1E_LDL',
        'G1E_CRTN', 'G1E_GFR', 'Q_SMK_YN', 'Q_DRK_FRQ_V09N', 'Q_DRK_AMT_V09N',
        'Q_PA_VD', 'Q_PA_MD', 'Q_PA_WALK', 'AF', 'CKD', 'COPD', 'DM', 'DVT',
        'DYS', 'HF', 'hyper', 'LD', 'MI', 'obesity', 'Obesity', 'stroke']]

y = df[['5year_survival_time(day)', '5year_death', '3year_survival_time(day)',
        '3year_death', '1year_survival_time(day)', '1year_death']]

In [9]:
categorical_columns = ['SEX','TCODE','MCODE_GRP','Q_SMK_YN','G1E_HGHT', 'G1E_WGHT', 'G1E_WSTC']

categorical_order_columns = ['AJCC7_STAGE','GRADE', 'G1E_URN_PROT']

onehot_encoder = OneHotEncoder(sparse=False)

X_encoded = pd.DataFrame(onehot_encoder.fit_transform(X[categorical_columns]))
X_encoded.columns = onehot_encoder.get_feature_names_out(categorical_columns)

# Drop unnecessary columns or rows from the dataset
X_continuous = X.drop(columns=categorical_columns)
X=pd.concat([X_continuous, X_encoded], axis=1)

label_encoder = LabelEncoder()

for column in categorical_order_columns:
    X[column]=label_encoder.fit_transform(X[column])

In [10]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(585, 66)
(585, 6)
(252, 66)
(252, 6)


In [12]:
# Display the first few rows of the dataset
X_train.head()

,GRADE,AGE,T_SIZE,AJCC7_STAGE,G1E_BMI,G1E_BP_SYS,G1E_BP_DIA,G1E_URN_PROT,G1E_HGB,G1E_FBS,...,G1E_WGHT_40,G1E_WGHT_50,G1E_WGHT_60,G1E_WGHT_70,G1E_WGHT_80,G1E_WGHT_90,G1E_WSTC_60,G1E_WSTC_70,G1E_WSTC_80,G1E_WSTC_90
234,2,30,150,5,21,129,81,0,12.6,94,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
220,2,41,15,2,21,90,62,0,11.8,90,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
316,2,50,18,0,24,120,80,0,12.7,107,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
333,1,49,24,0,26,110,70,0,14.8,92,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
132,2,29,15,0,20,111,69,0,14.5,102,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
y_train.columns

Index(['5year_survival_time(day)', '5year_death', '3year_survival_time(day)',
       '3year_death', '1year_survival_time(day)', '1year_death'],
      dtype='object')

In [14]:
X_train.to_csv('/project/kc20230915001/src/encoding_data/X_train.csv', index=False)
y_train.to_csv('/project/kc20230915001/src/encoding_data/y_train.csv', index=False)
X_test.to_csv('/project/kc20230915001/src/encoding_data/X_test.csv', index=False)
y_test.to_csv('/project/kc20230915001/src/encoding_data/y_test.csv', index=False)

### Dataset reconstruction

In [21]:

X_train = pd.read_csv('/project/kc20230915001/src/encoding_data/X_train.csv')

X_test = pd.read_csv('/project/kc20230915001/src/encoding_data/X_test.csv')

y_train = pd.read_csv('/project/kc20230915001/src/encoding_data/y_train.csv')

y_test = pd.read_csv('/project/kc20230915001/src/encoding_data/y_test.csv')

In [22]:
# Drop unnecessary columns or rows from the dataset
new_y_train = y_train.drop(columns=['3year_survival_time(day)', '3year_death', '1year_survival_time(day)','1year_death',])
new_y_test = y_test.drop(columns=['3year_survival_time(day)', '3year_death', '1year_survival_time(day)', '1year_death'])

In [23]:
new_y_train.to_csv('/project/kc20230915001/src/encoding_data/y_train_5year.csv', index=False)
new_y_test.to_csv('/project/kc20230915001/src/encoding_data/y_test_5year.csv', index=False)

### convert structured

In [84]:

X_train = pd.read_csv('/project/kc20230915001/src/encoding_data/X_train.csv')

X_test = pd.read_csv('/project/kc20230915001/src/encoding_data/X_test.csv')

y_train = pd.read_csv('/project/kc20230915001/src/encoding_data/y_train_5year.csv')

y_test = pd.read_csv('//project/kc20230915001/src/encoding_data/y_test_5year.csv')

In [85]:
y_train.columns

Index(['5year_survival_time(day)', '5year_death'], dtype='object')

In [86]:
new_order = ['5year_death', '5year_survival_time(day)']

In [87]:
y_train = y_train[new_order]
y_test = y_test[new_order]

In [88]:
y_train.columns

Index(['5year_death', '5year_survival_time(day)'], dtype='object')

In [89]:
def convert_to_structured_array(data):
    structured_data = np.zeros(data.shape[0], dtype={'names':('5year_death','5year_survival_time(day)'),'formats':('bool','f8')})
    structured_data['5year_death'] = data['5year_death'].astype(bool)
    structured_data['5year_survival_time(day)']=data['5year_survival_time(day)'].astype(float)
    return structured_data

y_train_structured = convert_to_structured_array(y_train)
y_test_structured = convert_to_structured_array(y_test)

In [90]:
print(y_train_structured[:5])

[(False, 1825.) (False, 1825.) (False, 1825.) (False, 1825.)
 (False, 1825.)]


### RSF

In [91]:
n_iterations = 10
all_results = []


for i in range(n_iterations):
    rsf = RandomSurvivalForest(n_estimators = 100, random_state = i)
# Fit the machine learning model to the training data
    rsf.fit(X_train, y_train_structured)
    
# Make predictions using the trained model
    y_pred = rsf.predict(X_test)
    c_index = concordance_index_censored(y_test_structured['5year_death'], y_test_structured['5year_survival_time(day)'], y_pred)[0]
    
    result = permutation_importance(rsf, X_test, y_test_structured, n_repeats=10, random_state=i)
    feature_importances = result.importances_mean
    features = X_train.columns
    
    for feature, importance in zip(features, feature_importances):
        all_results.append({
            "Iteration": i+1,
            "C-index": c_index,
            "feature": feature,
            "Importance": importance
        })
        
results_df = pd.DataFrame(all_results)
results_df.to_csv("C-index_and_feautre_importances_rsf_5year.csv", index=False)

mean_c_index = np.mean(c_index)
std_c_index = np.std(c_index)

print(f"C-index 평균: {mean_c_index:.4f}")
print(f"C-index 표준편차: {std_c_index:.4f}")

C-index 평균: 0.9282
C-index 표준편차: 0.0000


### GB-Survival

In [92]:
n_iterations = 10
all_results = []

for i in range(n_iterations):
    
    gb_model = GradientBoostingSurvivalAnalysis(n_estimators=100, random_state=i)
# Fit the machine learning model to the training data
    gb_model.fit(X_train, y_train_structured)
    
# Make predictions using the trained model
    y_pred = gb_model.predict(X_test)
    c_index = concordance_index_censored(y_test_structured['5year_death'], y_test_structured['5year_survival_time(day)'], y_pred)[0]
    
    
    feature_importances = gb_model.feature_importances_
    features = X_train.columns
    
    for feature, importance in zip(features, feature_importances):
        all_results.append({
            "Iteration": i+1,
            "C-index": c_index,
            "feature": feature,
            "Importance": importance
        })
        
results_df = pd.DataFrame(all_results)
results_df.to_csv("C-index_and_feautre_importances_gb_5year.csv", index=False)


mean_c_index = np.mean(c_index)
std_c_index = np.std(c_index)

print(f"C-index 평균: {mean_c_index:.4f}")
print(f"C-index 표준편차: {std_c_index:.4f}")

C-index 평균: 0.9357
C-index 표준편차: 0.0000


### EST

In [93]:
n_iterations = 10
all_results = []

for i in range(n_iterations):
    
    est_model = ExtraSurvivalTrees(n_estimators=100, random_state=i)
# Fit the machine learning model to the training data
    est_model.fit(X_train, y_train_structured)
    
# Make predictions using the trained model
    y_pred = est_model.predict(X_test)
    c_index = concordance_index_censored(y_test_structured['5year_death'], y_test_structured['5year_survival_time(day)'], y_pred)[0]
    
    result = permutation_importance(est_model, X_test, y_test_structured, n_repeats=10, random_state=i)
    feature_importances = result.importances_mean
    features = X_train.columns
    
    for feature, importance in zip(features, feature_importances):
        all_results.append({
            "Iteration": i+1,
            "C-index": c_index,
            "feature": feature,
            "Importance": importance
        })
        
results_df = pd.DataFrame(all_results)
results_df.to_csv("C-index_and_feautre_importances_est_5year.csv", index=False)


mean_c_index = np.mean(c_index)
std_c_index = np.std(c_index)

print(f"C-index 평균: {mean_c_index:.4f}")
print(f"C-index 표준편차: {std_c_index:.4f}")

C-index 평균: 0.9368
C-index 표준편차: 0.0000


### SVMSurvival

In [73]:
from sksurv.svm import FastSurvivalSVM
from sklearn.preprocessing import StandardScaler

n_iterations = 10
all_results = []

for i in range(n_iterations):
    
    scaler=StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    
    svm_model = FastSurvivalSVM(rank_ratio=0.5, random_state=i)
# Fit the machine learning model to the training data
    svm_model.fit(X_train_scaled, y_train_structured)
    
# Make predictions using the trained model
    y_pred = svm_model.predict(X_test_scaled)
    c_index = concordance_index_censored(y_test_structured['3year_death'], y_test_structured['3year_survival_time(day)'], y_pred)[0]
    
    
    feature_importances = svm_model.coef_
    features = X_train.columns
    
    for feature, importance in zip(features, feature_importances):
        all_results.append({
            "Iteration": i+1,
            "C-index": c_index,
            "feature": feature,
            "Importance": importance
        })
        
results_df = pd.DataFrame(all_results)
results_df.to_csv("C-index_and_feautre_importances_svm_3year.csv", index=False)


mean_c_index = np.mean(c_index)
std_c_index = np.std(c_index)

print(f"C-index 평균: {mean_c_index:.4f}")
print(f"C-index 표준편차: {std_c_index:.4f}")

C-index 평균: 0.0758
C-index 표준편차: 0.0000


### the most important feature

In [102]:
# Importing necessary libraries
importance_df = pd.read_csv('/project/kc20230915001/src/survival_results/C-index_and_feautre_importances_est_5year.csv')

feature_group = importance_df.groupby('feature').agg({'Importance':['mean','count']
                                                     }).reset_index()
feature_group.columns = ['Feature', 'Importance_mean', 'Importance_Count']

sorted_features = feature_group.sort_values('Importance_mean', ascending=False)

# Display the first few rows of the dataset
top_features = sorted_features.head(10)

top_features.to_csv('top_feautres_est_5year.csv', index=False)
print(top_features)

        Feature  Importance_mean  Importance_Count
2   AJCC7_STAGE         0.105182                10
62       T_SIZE         0.018127                10
37        GRADE         0.004572                10
63        hyper         0.003538                10
28  G1E_WGHT_50         0.002919                10
51   Q_SMK_YN_3         0.002542                10
16      G1E_HGB         0.002451                10
39           LD         0.002144                10
4          COPD         0.001855                10
14      G1E_GGT         0.001604                10
